In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('')
df

In [ ]:
x = list(df["message"])
y = list(df["label"])

In [ ]:
# make both list as numerics

In [ ]:
from sklearn.model_selection import train_test_split
xtrain , xtest , ytrain , ytest = train_test_split(x,y,test_size = 20, random_state = 0)

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("classla/xlm-roberta-base-multilingual-text-genre-classifier")

model = AutoModelForSequenceClassification.from_pretrained("classla/xlm-roberta-base-multilingual-text-genre-classifier")

In [ ]:
train_encodings = tokenizer(xtrain, truncation = True, padding = True)
test_encodeings = tokenizer(xtest, truncation = True, padding = True)